<a href="https://colab.research.google.com/github/toche7/LLM/blob/main/DataAnalysisGroq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Categorize Expense Data

### Test the LLM

In [ ]:
!pip -q install groq

In [ ]:
#=========VSCode=====================================================================
# from dotenv import find_dotenv, load_dotenv
# _ = load_dotenv(find_dotenv())
# import os
# secret_key = os.environ.get("GROQ_API_KEY")
#========google.colab======================================================================
from google.colab import userdata
secret_key = userdata.get('GROQ_API_KEY')

In [ ]:
from groq import Groq

clientGroq = Groq(
    api_key= secret_key,
)


In [ ]:
def get_completion(prompt, model= "llama3-70b-8192", clientGroq = clientGroq):
    messages = [{"role": "user", "content": prompt}]
    response = clientGroq.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        max_tokens= 4000,
        top_p = 0.5
    )
    return response.choices[0].message.content

### Import Data

In [ ]:
#!curl -o local_filename.csv https://raw.githubusercontent.com/toche7/LLM/main/transactions_2022_2023.csv

In [ ]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/toche7/LLM/main/transactions_2022_2023.csv")
#df = pd.read_csv("transactions_2022_2023.csv")
df.head()

In [ ]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

In [ ]:
unique_transactions[1:10]

### Categorize Transection with LLM

In [ ]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 5))
index_list

In [ ]:
def categorize_transactions(transaction_names):
    response = get_completion("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [ ]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam')

In [ ]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

In [ ]:
categories_df_all

In [ ]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

In [ ]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

In [ ]:
categories_df_all

In [ ]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '', regex=True)
categories_df_all

In [ ]:
# Plot the top 5 categories in pie chart of the categories_df_all dataframe
categories_df_all['Category'].value_counts().head(5).plot.pie(autopct='%1.1f%%')

import matplotlib.pyplot as plt

